In [3]:
import os
import cv2
from ultralytics import YOLO
import numpy as np

# --- CONFIGURAZIONE ---
labels_path = "34759_final_project_rect/seq_01/labels.txt"
frames_folder = "34759_final_project_rect/seq_01/image_02/data/"
calib_path = "34759_final_project_rect/calib_cam_to_cam.txt"

# --- 0) Carica YOLOv8s pre-addestrato ---
print("Caricamento modello YOLO...")
model = YOLO("yolo11l.pt")

# Mappa classi COCO → classi del progetto
class_map = {
    0: "Pedestrian",   # person
    1: "Bicycle"
}

# --- 1) Trova i frame disponibili ---
available_frames = sorted([int(f.split(".")[0]) for f in os.listdir(frames_folder) if f.endswith(".png")])
print(f"Totale frame disponibili: {len(available_frames)}")

# --- 2) Carica labels GT ---
labels = {}
with open(labels_path, "r") as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) < 15:
            continue
        frame = int(parts[0])
        obj_id = int(parts[1])
        cls = parts[2]
        xmin, ymin, xmax, ymax = map(float, parts[6:10])
        x, y, z = map(float, parts[13:16])
        if frame not in labels:
            labels[frame] = []
        labels[frame].append({"id": obj_id, "cls": cls, "bbox": (xmin, ymin, xmax, ymax), "xyz": (x, y, z)})

print(f"Labels caricate per {len(labels)} frame")

# --- Funzione per calcolare IoU ---
def IoU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) > 0 else 0

# --- 3) Ciclo sui frame ---
for frame_id in available_frames:
    img_path = os.path.join(frames_folder, f"{frame_id:06d}.png")
    img = cv2.imread(img_path)
    if img is None:
        print("ATTENZIONE: frame mancante:", img_path)
        continue
    h, w = img.shape[:2]

    # Disegna Ground Truth
    if frame_id in labels:
        for obj in labels[frame_id]:
            xmin, ymin, xmax, ymax = map(int, obj["bbox"])
            cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
            cv2.putText(img, f"GT {obj['cls']} ID:{obj['id']}", (xmin, max(0, ymin-8)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # YOLO detection
    results = model(img, verbose=False, device="cuda:0")[0]
    persons, bikes = [], []

    for box in results.boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        if conf < 0.6:
            continue
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        bbox = (x1, y1, x2, y2)
        if cls == 0: persons.append((bbox, conf))
        elif cls == 1: bikes.append((bbox, conf))

    # Riconoscimento Cyclist = solo quando c'è overlap tra persona e bici
    cyclists = []
    used_persons, used_bikes = set(), set()
    for i, (pb, pconf) in enumerate(persons):
        for j, (bb, bconf) in enumerate(bikes):
            if IoU(pb, bb) > 0.2:  # soglia IoU minima per considerare overlap
                x1 = min(pb[0], bb[0])
                y1 = min(pb[1], bb[1])
                x2 = max(pb[2], bb[2])
                y2 = max(pb[3], bb[3])
                cyclists.append(((x1, y1, x2, y2), max(pconf, bconf)))
                used_persons.add(i)
                used_bikes.add(j)

    # Disegna Cyclist
    for (x1, y1, x2, y2), conf in cyclists:
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 255), 2)
        cv2.putText(img, f"Cyclist {conf:.2f}", (x1, y1 - 8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)

    # Disegna persone NON ciclisti
    for idx, (pb, conf) in enumerate(persons):
        if idx in used_persons: continue
        x1, y1, x2, y2 = pb
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)
        cv2.putText(img, f"Pedestrian {conf:.2f}", (x1, y1 - 8),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 2)

    cv2.imshow("YOLO", img)
    key = cv2.waitKey(1)
    if key == 27: break

cv2.destroyAllWindows()


Caricamento modello YOLO...
Totale frame disponibili: 145
Labels caricate per 145 frame


In [2]:
# --- 1) Trova i frame disponibili ---
available_frames = sorted([int(f.split(".")[0]) for f in os.listdir(frames_folder) if f.endswith(".png")])
print(f"Totale frame disponibili: {len(available_frames)}")

# --- 2) Carica labels GT ---
labels = {}
with open(labels_path, "r") as file:
    for line in file:
        parts = line.strip().split()
        if len(parts) < 15: continue
        frame = int(parts[0])
        cls = parts[2]
        if cls != "Pedestrian": continue
        xmin, ymin, xmax, ymax = map(float, parts[6:10])
        if frame not in labels: labels[frame] = []
        labels[frame].append((xmin, ymin, xmax, ymax))

# --- 3) Inferenza YOLO e salvataggio predizioni ---
predictions_per_frame = {}

for frame_id in available_frames:
    img_path = os.path.join(frames_folder, f"{frame_id:06d}.png")
    img = cv2.imread(img_path)
    if img is None:
        print("Frame mancante:", img_path)
        continue

    results = model(img, verbose=False)[0]
    pred_boxes = []

    for box in results.boxes:
        cls = int(box.cls)
        conf = float(box.conf)
        if cls != 0 or conf < 0.6:  # solo pedestrians
            continue
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
        pred_boxes.append((x1, y1, x2, y2))

    predictions_per_frame[frame_id] = pred_boxes


Totale frame disponibili: 145


In [3]:
# --- Funzione IoU ---
def IoU(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    return interArea / float(boxAArea + boxBArea - interArea) if (boxAArea + boxBArea - interArea) > 0 else 0

iou_threshold = 0.5
total_TP = total_FP = total_FN = 0

for frame_id in predictions_per_frame:
    pred_boxes = predictions_per_frame[frame_id]
    gt_boxes = labels.get(frame_id, [])

    matched_gt = set()
    TP = FP = 0

    for pb in pred_boxes:
        matched = False
        for i, gt in enumerate(gt_boxes):
            if i in matched_gt: continue
            if IoU(pb, gt) >= iou_threshold:
                TP += 1
                matched_gt.add(i)
                matched = True
                break
        if not matched:
            FP += 1

    FN = len(gt_boxes) - len(matched_gt)

    total_TP += TP
    total_FP += FP
    total_FN += FN

    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0

    print(f"Frame {frame_id}: TP={TP}, FP={FP}, FN={FN}, Precision={precision:.2f}, Recall={recall:.2f}, F1={f1:.2f}")

# --- Metriche complessive ---
overall_precision = total_TP / (total_TP + total_FP) if total_TP + total_FP > 0 else 0
overall_recall = total_TP / (total_TP + total_FN) if total_TP + total_FN > 0 else 0
overall_f1 = 2 * overall_precision * overall_recall / (overall_precision + overall_recall) if overall_precision + overall_recall > 0 else 0

print(f"\nMetriche complessive: TP={total_TP}, FP={total_FP}, FN={total_FN}")
print(f"Precision={overall_precision:.2f}, Recall={overall_recall:.2f}, F1={overall_f1:.2f}")


Frame 0: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 1: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 2: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 3: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 4: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 5: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 6: TP=3, FP=0, FN=3, Precision=1.00, Recall=0.50, F1=0.67
Frame 7: TP=2, FP=0, FN=4, Precision=1.00, Recall=0.33, F1=0.50
Frame 8: TP=3, FP=0, FN=4, Precision=1.00, Recall=0.43, F1=0.60
Frame 9: TP=4, FP=0, FN=3, Precision=1.00, Recall=0.57, F1=0.73
Frame 10: TP=5, FP=0, FN=2, Precision=1.00, Recall=0.71, F1=0.83
Frame 11: TP=6, FP=0, FN=2, Precision=1.00, Recall=0.75, F1=0.86
Frame 12: TP=6, FP=0, FN=2, Precision=1.00, Recall=0.75, F1=0.86
Frame 13: TP=6, FP=0, FN=2, Precision=1.00, Recall=0.75, F1=0.86
Frame 14: TP=6, FP=1, FN=2, Precision=0.86, Recall=0.75, F1=0.80
Frame 15: TP=5, FP=1, FN=3, Precisi

In [4]:
left_folder = "34759_final_project_rect/seq_01/image_02/data/"
right_folder = "34759_final_project_rect/seq_01/image_03/data/"

# --- Carica calibrazione ---
def load_projection_matrices(calib_file):
    P = {}
    with open(calib_file, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("P_rect_"):
                key, values = line.split(":")
                cam = int(key.split("_")[-1])
                P[cam] = np.array(list(map(float, values.split()))).reshape(3,4)
    return P

P = load_projection_matrices(calib_path)
P_left = P[2]   # P_rect_02
P_right = P[3]  # P_rect_03

# Baseline in metri
f = P_left[0,0]               # focale in pixel
B = abs(P_right[0,3] / f)     # baseline = Tx / f
print(f"Focale: {f:.2f} px, Baseline: {B:.3f} m")

np.set_printoptions(precision=15, suppress=False)
print("P_left (02):\n", P_left)
print("P_right (03):\n", P_right)

Focale: 707.05 px, Baseline: 0.473 m
P_left (02):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02  4.575831e+01]
 [ 0.000000e+00  7.070493e+02  1.805066e+02 -3.454157e-01]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  4.981016e-03]]
P_right (03):
 [[ 7.070493e+02  0.000000e+00  6.040814e+02 -3.341081e+02]
 [ 0.000000e+00  7.070493e+02  1.805066e+02  2.330660e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  3.201153e-03]]


In [5]:
# --- Funzione per calcolare disparità ---
def compute_disparity(left_img, right_img):
    # Converti in scala di grigi
    gray_left = cv2.cvtColor(left_img, cv2.COLOR_BGR2GRAY)
    gray_right = cv2.cvtColor(right_img, cv2.COLOR_BGR2GRAY)

    # StereoSGBM parameters
    min_disp = 0
    num_disp = 128  # deve essere multiplo di 16
    block_size = 7

    stereo = cv2.StereoSGBM_create(
        minDisparity=min_disp,
        numDisparities=num_disp,
        blockSize=block_size,
        P1=8*3*block_size**2,
        P2=32*3*block_size**2,
        uniquenessRatio=10,
        speckleWindowSize=100,
        speckleRange=32,
        disp12MaxDiff=1,
        preFilterCap=63
    )
    
    disp = stereo.compute(gray_left, gray_right).astype(np.float32) / 16.0
    return disp


In [11]:
# --- Funzione robusta per estrarre coordinate 3D ---
def bbox_to_3d(bbox, disp_map, f, B, P_left, window=7):
    x1, y1, x2, y2 = map(int, bbox)
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)

    h, w = disp_map.shape
    half = window // 2

    # Finestra attorno al centro del bbox
    x0 = max(0, cx - half)
    x1w = min(w, cx + half + 1)
    y0 = max(0, cy - half)
    y1w = min(h, cy + half + 1)

    patch = disp_map[y0:y1w, x0:x1w].flatten()
    valid = patch[np.isfinite(patch) & (patch > 0)]

    if len(valid) == 0:
        return None

    disp = np.median(valid)
    if disp <= 0 or not np.isfinite(disp):
        return None

    Z = f * B / disp
    X = (cx - P_left[0,2]) * Z / f
    Y = (cy - P_left[1,2]) * Z / f

    return X, Y, Z

In [7]:
frame_id = 0  # scegli il frame che vuoi analizzare


# --- Funzione per caricare le labels con posizione 3D ---
def load_labels(labels_path, frame_id, object_type="Pedestrian"):
    labels = []
    with open(labels_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != frame_id:
                continue
            if parts[2] != object_type:
                continue
            xmin, ymin, xmax, ymax = map(float, parts[6:10])     # bbox rettificata
            x, y, z = map(float, parts[13:16])                   # coordinate 3D
            labels.append({"bbox": (xmin, ymin, xmax, ymax), "xyz": (x, y, z)})
    return labels



# --- Carica immagini stereo ---
left_img = cv2.imread(os.path.join(left_folder, f"{frame_id:06d}.png"))
right_img = cv2.imread(os.path.join(right_folder, f"{frame_id:06d}.png"))

if left_img is None or right_img is None:
    raise RuntimeError(f"Immagini stereo mancanti per frame {frame_id:06d}")


# --- Calcola disparity map ---
disp_map = compute_disparity(left_img, right_img)

print("Disparity map stats:",
      "min =", np.min(disp_map),
      "max =", np.max(disp_map),
      "valid % =", np.count_nonzero(disp_map > 0) / disp_map.size * 100)



# --- Carica GT del frame ---
gt_objects = load_labels(labels_path, frame_id)

# !!! --- Usa le predizioni del frame corretto (FIX DEL TUO BUG) ---
pred_boxes = predictions_per_frame.get(frame_id, [])
if len(pred_boxes) == 0:
    print(f"⚠️ Nessuna predizione trovata per frame {frame_id}")


matched_pred_idx = set()

print(f"{'GT_X':>7} {'GT_Y':>7} {'GT_Z':>7} | {'Pred_X':>7} {'Pred_Y':>7} {'Pred_Z':>7} | IoU")
print("-"*70)


# --- Matching GT <-> predizioni ---
for gt in gt_objects:
    gt_bbox = gt["bbox"]
    gt_xyz = gt["xyz"]

    best_iou = 0
    best_pred = None
    best_idx = -1

    for i, pb in enumerate(pred_boxes):
        if i in matched_pred_idx:
            continue

        iou = IoU(gt_bbox, pb)
        if iou > best_iou:
            best_iou = iou
            best_pred = pb
            best_idx = i

    # Converti la predizione in 3D
    if best_pred is not None:
        matched_pred_idx.add(best_idx)
        pred_xyz = bbox_to_3d(best_pred, disp_map, f, B, P_left)

        if pred_xyz is not None:
            px, py, pz = pred_xyz
        else:
            px = py = pz = float('nan')
    else:
        px = py = pz = float('nan')
        best_iou = 0.0

    print(f"{gt_xyz[0]:7.2f} {gt_xyz[1]:7.2f} {gt_xyz[2]:7.2f} | "
          f"{px:7.2f} {py:7.2f} {pz:7.2f} | {best_iou:.2f}")


Disparity map stats: min = -1.0 max = 127.0 valid % = 79.81672849319908
   GT_X    GT_Y    GT_Z |  Pred_X  Pred_Y  Pred_Z | IoU
----------------------------------------------------------------------
  -0.88    1.37    6.82 |   -0.81    0.49    5.95 | 0.88
  -1.33    1.40    5.92 |   -1.23    0.56    5.33 | 0.93
  -3.44    0.60   21.98 |     nan     nan     nan | 0.00
  -2.64    0.41   26.11 |     nan     nan     nan | 0.00
  -1.91    0.36   25.90 |     nan     nan     nan | 0.00
  -1.08    0.38   25.06 |     nan     nan     nan | 0.00


In [8]:
import cv2
import numpy as np
import os

frame_id = 0

# --- Percorsi ---
left_folder = "34759_final_project_rect/seq_01/image_02/data/"
right_folder = "34759_final_project_rect/seq_01/image_03/data/"
labels_path = "34759_final_project_rect/seq_01/labels.txt"

# --- Carica labels GT per il frame ---
def load_labels(labels_path, frame_id, object_type="Pedestrian"):
    labels = []
    with open(labels_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != frame_id:
                continue
            if parts[2] != object_type:
                continue
            xmin, ymin, xmax, ymax = map(float, parts[6:10])
            labels.append((xmin, ymin, xmax, ymax))
    return labels

gt_boxes = load_labels(labels_path, frame_id)

# --- Carica immagini ---
left_img = cv2.imread(os.path.join(left_folder, f"{frame_id:06d}.png"))

# --- Disegna GT (verde) e predizioni YOLO (rosso) ---
vis_img = left_img.copy()

# Ground Truth
for bbox in gt_boxes:
    x1, y1, x2, y2 = map(int, bbox)
    cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(vis_img, "GT", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

# YOLO Predizioni
for bbox in pred_boxes:
    x1, y1, x2, y2 = map(int, bbox)
    cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0, 0, 255), 2)
    cv2.putText(vis_img, "YOLO", (x1, y1-5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

# --- Mostra immagine ---
cv2.imshow(f"Frame {frame_id}", vis_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [18]:
import cv2
import numpy as np
import os

frame_id = 0

# --- Percorsi ---
left_folder = "34759_final_project_rect/seq_01/image_02/data/"
right_folder = "34759_final_project_rect/seq_01/image_03/data/"
labels_path = "34759_final_project_rect/seq_01/labels.txt"

# --- Funzione per proiettare 3D -> immagine ---
def project_point(P, X, Y, Z):
    pt = np.array([X, Y, Z, 1.0])
    uvw = P @ pt
    u = uvw[0] / uvw[2]
    v = uvw[1] / uvw[2]
    return int(u), int(v)

# --- Carica labels GT + 3D ---
def load_labels_with_xyz(labels_path, frame_id):
    objs = []
    with open(labels_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if int(parts[0]) != frame_id: continue
            if parts[2] != "Pedestrian": continue
            xmin, ymin, xmax, ymax = map(float, parts[6:10])
            x, y, z = map(float, parts[13:16])
            objs.append({"bbox": (xmin,ymin,xmax,ymax), "xyz": (x,y,z)})
    return objs

gt_objects = load_labels_with_xyz(labels_path, frame_id)

# --- Carica immagini ---
left_img = cv2.imread(os.path.join(left_folder, f"{frame_id:06d}.png"))
vis_img = left_img.copy()

# ========== DISEGNA GT (verde) ==========
for obj in gt_objects:
    x1, y1, x2, y2 = map(int, obj["bbox"])
    cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0,255,0), 2)
    cv2.putText(vis_img, "GT", (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

# ========== DISEGNA YOLO (rosso) ==========
for (x1, y1, x2, y2) in pred_boxes:
    cv2.rectangle(vis_img, (x1, y1), (x2, y2), (0,0,255), 2)
    cv2.putText(vis_img, "YOLO", (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

# ============================================================
# 🔵🔵🔵  PROIEZIONE 3D → 2D (punti blu per GT e Pred)  🔵🔵🔵
# ============================================================

# --- Proietta GT in blu ---
for obj in gt_objects:
    X, Y, Z = obj["xyz"]
    u, v = project_point(P_left, X, Y, Z)
    if 0 <= u < vis_img.shape[1] and 0 <= v < vis_img.shape[0]:
        cv2.circle(vis_img, (u, v), 5, (255,0,0), -1)  # blu pieno
        cv2.putText(vis_img, "GT 3D", (u+5, v),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,0,0), 2)

# --- Proietta predizioni YOLO in blu chiaro ---
for gt, idx in zip(gt_objects, matched_pred_idx):
    pred_bbox = pred_boxes[idx]
    pred_xyz = bbox_to_3d(pred_bbox, disp_map, f, B, P_left)

    if pred_xyz is not None:
        px, py, pz = pred_xyz
        u, v = project_point(P_left, px, py, pz)
        if 0 <= u < vis_img.shape[1] and 0 <= v < vis_img.shape[0]:
            cv2.circle(vis_img, (u, v), 5, (255,200,0), -1)  # giallo-azzurro
            cv2.putText(vis_img, "Pred 3D", (u+5, v),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255,200,0), 2)

# --- Mostra immagine finale ---
cv2.imshow("GT, YOLO, and 3D Points", vis_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [20]:
import numpy as np
import cv2

# -------------------------------------------------------------
# 1) Robust bottom-center 3D estimation using disparity map
# -------------------------------------------------------------
def estimate_bottom_center_3d_from_disp(bbox, disp_map, P_left, P_right, min_valid=5):
    x1, y1, x2, y2 = bbox
    H, W = disp_map.shape

    # bottom-center pixel del bbox
    u = int((x1 + x2) / 2)
    v = int(y2 - 1)

    # patch per robustezza (15% altezza bbox)
    h = y2 - y1
    w = x2 - x1
    patch_h = max(3, int(h * 0.15))
    patch_w = max(5, int(w * 0.30))

    x1p = max(0, u - patch_w//2)
    x2p = min(W-1, u + patch_w//2)
    y1p = max(0, v - patch_h//2)
    y2p = min(H-1, v + patch_h//2)

    patch = disp_map[y1p:y2p+1, x1p:x2p+1]
    valid = patch[(patch > 0) & np.isfinite(patch)]

    if len(valid) < min_valid:
        return None

    disparity = float(np.median(valid))

    # Camera parameters
    f = P_left[0,0]
    cx = P_left[0,2]
    cy = P_left[1,2]
    B = abs(P_right[0,3] / f)

    Z = f * B / disparity
    X = (u - cx) * Z / f
    Y = (v - cy) * Z / f

    return (X, Y, Z)

# -------------------------------------------------------------
# 2) Projection 3D → pixels
# -------------------------------------------------------------
def project_point(P, X, Y, Z):
    pt = np.array([X, Y, Z, 1])
    uvw = P @ pt
    u = uvw[0] / uvw[2]
    v = uvw[1] / uvw[2]
    return int(u), int(v)

# -------------------------------------------------------------
# 3) Visualizza tutto
# -------------------------------------------------------------
vis = left_img.copy()

# ────────────────────────────────────────────────
# YOLO BOUNDING BOXES
# ────────────────────────────────────────────────
for (x1, y1, x2, y2) in pred_boxes:
    cv2.rectangle(vis, (x1,y1), (x2,y2), (0,0,255), 2)
    cv2.putText(vis, "YOLO", (x1, y1-5),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 2)

# ────────────────────────────────────────────────
# 2) GT POINTS (3D → 2D proiettati)
# ────────────────────────────────────────────────
for obj in gt_objects:
    Xg, Yg, Zg = obj["xyz"]
    ug, vg = project_point(P_left, Xg, Yg, Zg)
    cv2.circle(vis, (ug,vg), 6, (0,255,255), -1)  # YELLOW
    cv2.putText(vis, f"GT", (ug+5,vg),
                cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,255),2)

# ────────────────────────────────────────────────
# 3) STEREO 3D POINTS (bottom-center estimation)
# ────────────────────────────────────────────────
for bbox in pred_boxes:
    xyz = estimate_bottom_center_3d_from_disp(bbox, disp_map, P_left, P_right)

    if xyz is None:
        continue

    Xp, Yp, Zp = xyz
    up, vp = project_point(P_left, Xp, Yp, Zp)

    cv2.circle(vis, (up,vp), 6, (255,0,0), -1)  # BLUE
    cv2.putText(vis, f"Pred", (up+5,vp),
                cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,0,0),2)

# -------------------------------------------------------------
# SHOW
# -------------------------------------------------------------
cv2.imshow("YOLO (red), GT Point (yellow), Stereo Point (blue)", vis)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
def estimate_bottom_center_3d_from_disp(bbox, disp_map, P_left, min_valid=5, patch_h_ratio=0.18, patch_w=5):
    """
    bbox: (x1,y1,x2,y2) in px
    disp_map: disparity (float, pixels), invalid <=0
    P_left: 3x4
    returns (X,Y,Z) or None if non valido
    """
    x1,y1,x2,y2 = bbox
    h = y2 - y1
    w = x2 - x1
    # bottom-center pixel (float)
    u = (x1 + x2) / 2.0
    v = y2 - 1  # preferire leggermente sopra il bordo estremo, evita fuori immagine

    # patch: una striscia rettangolare nella parte inferiore del bbox
    patch_h = max(3, int(round(h * patch_h_ratio)))  # es. 18% dell'altezza
    patch_w = max(3, int(round(w * 0.2)))  # 20% larghezza o un minimo
    # centro patch sul bottom-center
    x0 = int(round(max(0, min(disp_map.shape[1]-1, u))))
    y0 = int(round(max(0, min(disp_map.shape[0]-1, v - patch_h//2))))
    x1p = max(0, x0 - patch_w//2)
    x2p = min(disp_map.shape[1]-1, x0 + patch_w//2)
    y1p = max(0, y0)
    y2p = min(disp_map.shape[0]-1, y0 + patch_h - 1)

    patch = disp_map[y1p:y2p+1, x1p:x2p+1]
    valid = patch[np.isfinite(patch) & (patch > 0)]
    # if not enough valid disparity, expand search (coarse)
    if valid.size < min_valid:
        # try larger box: full lower half
        y1p = int(max(0, y1 + h//2))
        y2p = int(min(disp_map.shape[0]-1, y2))
        x1p = int(max(0, x1))
        x2p = int(min(disp_map.shape[1]-1, x2))
        patch = disp_map[y1p:y2p+1, x1p:x2p+1]
        valid = patch[np.isfinite(patch) & (patch > 0)]

    if valid.size < min_valid:
        return None

    d_med = float(np.median(valid))

    f = P_left[0,0]
    cx = P_left[0,2]
    cy = P_left[1,2]
    B = abs(P_right[0,3] / f)  # usa P_right definita globalmente nel tuo notebook

    Z = f * B / d_med
    X = (u - cx) * Z / f
    Y = (v - cy) * Z / f
    return (X, Y, Z)